In [2]:
import numpy as np
import matplotlib.pyplot as plt
num_of_points = 2
num_symbols = 10000
noise_power = 0.3
phase_noise=0.5
#control data for the loops
num_epochs= 1
learning_rate=0.1
n = (np.random.randn(num_symbols) + 1j*np.random.randn(num_symbols))/np.sqrt(2)

In [3]:
def labels_creation(num_of_points):
    base=0
    possible_labels = []
    for l in range(0,num_of_points):
        if l%2 == 0:
            possible_labels.append(1 + base * 2)
        else:
            possible_labels.append(-1 - base * 2)
            base=base+1
    return possible_labels

In [4]:
def plot_grapgh(x, y):
    colors = ['red' if label == 1 else 'blue' for label in y]
    plt.figure(figsize=(5, 5))
    plt.scatter(np.real(x), np.imag(x), c=colors, alpha=0.5)
    plt.title("BPSK Constellation")
    plt.xlabel("In-phase (I)")
    plt.ylabel("Quadrature (Q)")
    plt.grid(True)
    plt.axis('equal')  # Keep aspect ratio
    plt.show()

In [5]:
def labels_insetion(num_of_points,data):
    labels=labels_creation(num_of_points)
    identification_of_lables_in_data = []

    reference_symbol = []
    for x in range(num_of_points):
        x_degrees = (x*360/num_of_points +  0) 
        x_radians = x_degrees*np.pi/180.0
        reference_symbol.append(np.cos(x_radians) + 1j*np.sin(x_radians))
    
    for d in data:
        for i in range(0,num_of_points):
            if np.isclose(d.real, reference_symbol[i].real, atol=1e-20) and \
               np.isclose(d.imag, reference_symbol[i].imag, atol=1e-20):
                identification_of_lables_in_data.append(labels[i])
                break

    return identification_of_lables_in_data
         

In [6]:
def data_separation(data):
    data = np.array(data)
    separated = np.stack((np.real(data), np.imag(data)), axis=1)
    return separated

In [8]:
def create_data_set_normal( phase_noise = 0 , noise_power = 0):
    np.random.seed(4)
    # AWGN with unity power
    x_int = np.random.randint(0, num_of_points, num_symbols)
    x_degrees = (x_int*360/num_of_points +  0)#np.random.randint(0, 360)) % 360# 45, 135, 225, 315 degree
    x_radians = x_degrees*np.pi/180.0 # sin() and cos() takes in radians
    x_symbols = (np.cos(x_radians) + 1j*np.sin(x_radians))* np.exp(1j*phase_noise) + n * np.sqrt(noise_power)
    return x_symbols

In [9]:
Data = create_data_set_normal()
Data_with_noise = create_data_set_normal(noise_power = 0.1 ,phase_noise = 0)
Data_with_phase = create_data_set_normal(phase_noise = 0.1,noise_power = 0)
Data_with_phase_and_noise = create_data_set_normal(phase_noise = 0,noise_power = 0)
Target = labels_insetion(num_of_points, Data)

#plot_grapgh(Data,Target)
#plot_grapgh(Data_with_noise,Target)
#plot_grapgh(Data_with_phase,Target)
#plot_grapgh(Data_with_phase_and_noise,Target)

In [10]:
#normal creation
#x_int = np.random.randint(0, num_of_points, num_symbols) 
#x_degrees = (x_int*360/num_of_points +  0)#np.random.randint(0, 360)) % 360# 45, 135, 225, 315 degrees
#x_radians = x_degrees*np.pi/180.0 # sin() and cos() takes in radians
#x_symbols = np.cos(x_radians) + 1j*np.sin(x_radians) # this produces our QPSK complex symbols
#AWGN addition
#n = (np.random.randn(num_symbols) + 1j*np.random.randn(num_symbols))/np.sqrt(2) # AWGN with unity power
#r_noise = x_symbols + n * np.sqrt(noise_power)
#addition of Phase noise

#phase_noise = np.random.randn(len(x_symbols)) * phase_noise
#r_phase = x_symbols * np.exp(1j*phase_noise)
#AWGN with Phase noise

#r = x_symbols * np.exp(1j*phase_noise) + n * np.sqrt(noise_power)
#plots
#Target_for_given_data = labels_insetion(num_of_points,x_symbols)
#plot_grapgh(x_symbols,Target_for_given_data)
#plot_grapgh(r_noise,Target_for_given_data)
#plot_grapgh(r_phase,Target_for_given_data)
#plot_grapgh(r,Target_for_given_data)

In [11]:
#only works for 2 points since perceptron is not made to detect non linearly separable, i woild need a multlayer perceptron
def activation_function_perceptron(y):
    if y <= 0:
        return -1
    else :
        return 1

In [12]:
def test_training(testing_data , weights , target , error=0):
    testing_data=data_separation(testing_data)
    for i, x in enumerate(testing_data):
        v = np.dot(weights, x)
        y = activation_function_perceptron(v)
        if(target[i] != y):
           error= error+1
    return error

In [13]:
# this is all the data i need to have : hte Target or desire that i have already, 
#the data itself that can be generated at any moment and the weights that tipically are inicisted at zeros, 
#soo i have everityin needed 
Target_for_given_data = labels_insetion(num_of_points,Data)
new_data = data_separation(Data)
weights = np.zeros(2)
#training
for i in range(num_epochs):
    for x, target in zip(new_data, Target_for_given_data):
        v = np.dot(weights.T, x)
        y = activation_function_perceptron(v)
        weights = weights + learning_rate * (target - y)*x

In [14]:
#tests with noise 
error= test_training(Data , weights ,Target_for_given_data)
print('pure=',error)
error1= test_training(Data_with_noise ,weights ,Target_for_given_data)
print('noice=',error1)
error2= test_training(Data_with_phase,weights,Target_for_given_data)
print('phase=',error2)
error3= test_training(Data_with_phase_and_noise,weights,Target_for_given_data)
print('both=',error3)

pure= 0
noice= 0
phase= 0
both= 0
